In [95]:
import pandas as pd
import numpy as np

In [166]:
da_univ = pd.read_csv('DA University&College List - Ct_nh_ma_schools_DA - Sheet1.csv')

In [167]:
da_univ.drop(da_univ.columns.difference(['INSTNM', 'LATITUDE', 'LONGITUDE', 'ZIP', 'CITY', 'STABBR']), 1, inplace=True)





In [181]:
da_store = pd.read_csv('DA_res_list - Sheet1.csv')

In [222]:

da_store = da_store.dropna(subset=['Address'])
da_store.drop(da_store.columns.difference(['Restaurant #', 'Restaurant Name', 'Address', 'City', 'State', 'Zip Code']), 1, inplace=True)
da_store.reset_index(drop=True)

,Restaurant #,Restaurant Name,Address,City,State,Zip Code
0,3006,Weymouth Dual,"Riverway Plaza, Weymouth, MA",Weymouth,MA,2191
1,3009,Somerset Dual,"Somerset Plaza, Rt. 6, Somerset, MA",Somerset,MA,2725
2,3502,Westboro Mass Pike West,"Mile Post 105; Mass Turnpike W., Westboro, MA",Westboro,MA,1581
3,3503,Charlton Mass Pike East,"Mile Post 81; Mass Turnpike E., Charlton, MA",Charlton,MA,1507
4,3504,Charlton Mass Pike West,"Mile Post 89; Mass Turnpike W., Charlton City, MA",Charlton City,MA,1508
5,3505,Natick Travel Plaza (Mass Pike),"Mile Post 117, Natick, MA",Natick,MA,1760
6,5018,Wethersfield,"1345 Silas Deane Hwy., Rt 99, Wethersfield, CT",Wethersfield,CT,6109
7,5019,Keene,"189 West St., Keene, NH",Keene,NH,3431
8,5021,Cranston,"875 Reservoir Ave., Rt. 2, Cranston, RI",Cranston,RI,2910
9,5022,Southington,"641 Queen St., Rt. 10, Southington, CT",Southington,CT,6489


### Get the address of da_univ from lat long coordinates.

In [223]:
da_univ = da_univ.dropna(subset=['LATITUDE', 'LONGITUDE'])


In [224]:
cols = ['Address', 'City', 'State']
da_store['Address'] = da_store[cols].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)
da_store = da_store.reset_index(drop = True)
#da_store.head()

In [225]:
from opencage.geocoder import OpenCageGeocode


In [226]:
key = 'e48659b4500a4db7bf6fa31641f8b688'  # get api key from:  https://opencagedata.com



geocoder = OpenCageGeocode(key)




query = 'Denver, Colorado'  




results = geocoder.geocode(query)




#print (results)

In [227]:
lat = results[0]['geometry']['lat']

lng = results[0]['geometry']['lng']

print (lat, lng)

39.7392364 -104.9848623


In [228]:
list_lat = []   # create empty lists

list_long = []

for index, row in da_store.iterrows(): # iterate over rows in dataframe


    Address = row['Address']
    query = str(Address)

    results = geocoder.geocode(query)   
    lat = results[0]['geometry']['lat']
    long = results[0]['geometry']['lng']

    list_lat.append(lat)
    list_long.append(long)

# create new columns from lists    

da_store['lat'] = list_lat   

da_store['lon'] = list_long

In [132]:
da_store.head(85)
export_csv = da_store.to_csv('da_store.csv')

In [113]:
da_store.shape

(83, 12)

In [82]:
da_univ.shape

(85, 6)

In [120]:
from sklearn.neighbors import NearestNeighbors

from math import radians, cos, sin, asin, sqrt
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [233]:
list_store = []
dist = []


ndx_stores = 0
for index_store, row_store in da_store.iterrows():
    store_lat = row_store['lat']
    store_lon = row_store['lon']
    
    ndx_univ = 0
    for index_univ, row_univ in da_univ.iterrows():
        univ_lat = row_univ['LATITUDE']
        univ_lon = row_univ['LONGITUDE']
        distance[ndx_univ] = haversine_np(store_lon, store_lat, univ_lon, univ_lat)
        ndx_univ += 1     
    
    list_store[ndx_stores] = min(distance)
    

#points = haversine_np(da_store['lon'], da_store['lat'], da_univ['LONGITUDE'], da_univ['LATITUDE'])

#print(points)
#points = points.reshape(1, -1)
#haversine_np(-97.441113,35.221090,-97.520280,35.465438)
#nbrs = NearestNeighbors(n_neighbors=2, metric=distance).fit(points)

#distances, indices = nbrs.kneighbors(points)

#result = distances[:, 1]
#da_store.head()

TypeError: 'numpy.float64' object does not support item assignment

In [133]:
export_csv2 = da_univ.to_csv('da_univ.csv')

In [189]:
%qtconsole

In [239]:
# Build an index that contain every pairing of Store - University
idx = pd.MultiIndex.from_product([da_store.index, da_univ.index], names=['Store', 'Univ'])

# Pull the coordinates of the store and the universities together
# We don't need their name here
df = pd.DataFrame(index=idx) \
        .join(da_store[['lat', 'lon']], on='Store') \
        .join(da_univ[['LATITUDE', 'LONGITUDE']], on='Univ')

df['Distance'] = haversine_np(*df[['lat', 'lon', 'LATITUDE', 'LONGITUDE']].values.T)

# The closest university to each store
min_distance = df.loc[df.groupby('Store')['Distance'].idxmin(), 'Distance']

# Pulling everything together
df_comp = min_distance.to_frame().join(da_store, on='Store').join(da_univ, on='Univ') \
    [['Restaurant Name', 'INSTNM', 'Distance']]


df_comp.head()

,,Restaurant Name,INSTNM,Distance
Store,Univ,,,
0,32,Weymouth Dual,Massachusetts School of Barbering,7.274241
1,68,Somerset Dual,Salter School-Fall River,1.303253
2,66,Westboro Mass Pike West,Spa Tech Institute-Westboro,3.288097
3,2,Charlton Mass Pike East,Assumption College,13.678788
4,35,Charlton Mass Pike West,Mount Wachusett Community College,17.301768


In [240]:
ex = df_comp.to_csv('df_comp.csv')